In [79]:
import pandas as pd


In [80]:
pip install lxml


Note: you may need to restart the kernel to use updated packages.


In [81]:
df=pd.read_html("https://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13",header=0)[0]

In [82]:
df["종목코드"]

0         4840
1       155660
2         1250
3       294870
4        92220
         ...  
2443    189350
2444    199800
2445    222670
2446    331660
2447    212310
Name: 종목코드, Length: 2448, dtype: int64

In [83]:
pip install html5lib


Note: you may need to restart the kernel to use updated packages.


In [84]:
pip install BeautifulSoup4

Note: you may need to restart the kernel to use updated packages.


In [85]:
import requests
from bs4 import BeautifulSoup

In [139]:
from bs4 import BeautifulSoup
import requests
import urllib


headers = {'User-Agent': 'Mozilla/5.0'}

span_list=[]
for i in range(1,10):
    response = requests.get( "https://finance.naver.com/item/sise_day.nhn?code=004840&page={0}".format(i), headers=headers)

    soup=BeautifulSoup(response.text, "html.parser")
    stock_list=soup.select("span")
    
    span_list.extend(stock_list[1:])




print(span_list)

[<span class="tah p10 gray03">2021.06.17</span>, <span class="tah p11">8,040</span>, <span class="tah p11 nv01">
				50
				</span>, <span class="tah p11">8,030</span>, <span class="tah p11">8,130</span>, <span class="tah p11">8,000</span>, <span class="tah p11">21,979</span>, <span class="tah p10 gray03">2021.06.16</span>, <span class="tah p11">8,090</span>, <span class="tah p11 red02">
				90
				</span>, <span class="tah p11">8,080</span>, <span class="tah p11">8,090</span>, <span class="tah p11">7,970</span>, <span class="tah p11">46,875</span>, <span class="tah p10 gray03">2021.06.15</span>, <span class="tah p11">8,000</span>, <span class="tah p11 nv01">
				240
				</span>, <span class="tah p11">8,220</span>, <span class="tah p11">8,220</span>, <span class="tah p11">7,950</span>, <span class="tah p11">141,253</span>, <span class="tah p10 gray03">2021.06.14</span>, <span class="tah p11">8,240</span>, <span class="tah p11 nv01">
				240
				</span>, <span class="tah p11">8,600</spa

In [125]:
import numpy as np

In [148]:
# pd.DataFrame(span_list,columns=["날짜","종가","전일비","시가","고가","저가","거래량"])
pd.DataFrame(span_list)
# sp=np.array(span_list)
# len(sp)

# sp=sp.reshape(,-1)
# sp

,0
0,2021.06.17
1,"8,040"
2,\n\t\t\t\t50\n\t\t\t\t
3,"8,030"
4,"8,130"
...,...
625,\n\t\t\t\t70\n\t\t\t\t
626,"7,110"
627,"7,210"
628,"7,100"


In [1]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jan 13 17:11:27 2021

@author: User
"""

import pandas as pd
from bs4 import BeautifulSoup
import pymysql, calendar, time, json
import requests
from datetime import datetime
from threading import Timer


class DBUpdater:  
    def __init__(self):
        """생성자: MariaDB 연결 및 종목코드 딕셔너리 생성"""
        self.conn = pymysql.connect(host='localhost', user='root',
            password='up318bin473!', db='INVESTAR', charset='utf8')
        
        with self.conn.cursor() as curs:
            sql = """
            CREATE TABLE IF NOT EXISTS company_info (
                code VARCHAR(20),
                company VARCHAR(40),
                last_update DATE,
                PRIMARY KEY (code))
            """
            curs.execute(sql)
            sql = """
            CREATE TABLE IF NOT EXISTS daily_price (
                code VARCHAR(20),
                date DATE,
                open BIGINT(20),
                high BIGINT(20),
                low BIGINT(20),
                close BIGINT(20),
                diff BIGINT(20),
                volume BIGINT(20),
                PRIMARY KEY (code, date))
            """
            curs.execute(sql)
        self.conn.commit()
        self.codes = dict()
               
    def __del__(self):
        """소멸자: MariaDB 연결 해제"""
        self.conn.close() 
     
    def read_krx_code(self):
        """KRX로부터 상장기업 목록 파일을 읽어와서 데이터프레임으로 반환"""
        url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method='\
            'download&searchType=13'
        krx = pd.read_html(url, header=0)[0]
        krx = krx[['종목코드', '회사명']]
        krx = krx.rename(columns={'종목코드': 'code', '회사명': 'company'})
        krx.code = krx.code.map('{:06d}'.format)
        return krx
    
    def update_comp_info(self):
        """종목코드를 company_info 테이블에 업데이트 한 후 딕셔너리에 저장"""
        sql = "SELECT * FROM company_info"
        df = pd.read_sql(sql, self.conn)
        for idx in range(len(df)):
            self.codes[df['code'].values[idx]] = df['company'].values[idx]
                    
        with self.conn.cursor() as curs:
            sql = "SELECT max(last_update) FROM company_info"
            curs.execute(sql)
            rs = curs.fetchone()
            today = datetime.today().strftime('%Y-%m-%d')
            if rs[0] == None or rs[0].strftime('%Y-%m-%d') < today:
                krx = self.read_krx_code()
                for idx in range(len(krx)):
                    code = krx.code.values[idx]
                    company = krx.company.values[idx]                
                    sql = f"REPLACE INTO company_info (code, company, last"\
                        f"_update) VALUES ('{code}', '{company}', '{today}')"
                    curs.execute(sql)
                    self.codes[code] = company
                    tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')
                    print(f"[{tmnow}] #{idx+1:04d} REPLACE INTO company_info "\
                        f"VALUES ({code}, {company}, {today})")
                self.conn.commit()
                print('')              

    def read_naver(self, code, company, pages_to_fetch):
        """네이버에서 주식 시세를 읽어서 데이터프레임으로 반환"""
        try:
            url = f"http://finance.naver.com/item/sise_day.nhn?code={code}"
            html = BeautifulSoup(requests.get(url,
                headers={'User-agent': 'Mozilla/5.0'}).text, "lxml")
            pgrr = html.find("td", class_="pgRR")
            if pgrr is None:
                return None
            s = str(pgrr.a["href"]).split('=')
            lastpage = s[-1] 
            df = pd.DataFrame()
            pages = min(int(lastpage), pages_to_fetch)
            for page in range(1, pages + 1):
                pg_url = '{}&page={}'.format(url, page)
                df = df.append(pd.read_html(requests.get(pg_url,
                    headers={'User-agent': 'Mozilla/5.0'}).text)[0])                                          
                tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')
              
            df = df.rename(columns={'날짜':'date','종가':'close','전일비':'diff'
                ,'시가':'open','고가':'high','저가':'low','거래량':'volume'})
            df['date'] = df['date'].replace('.', '-')
            df = df.dropna()
            df[['close', 'diff', 'open', 'high', 'low', 'volume']] = df[['close',
                'diff', 'open', 'high', 'low', 'volume']].astype(int)
            df = df[['date', 'open', 'high', 'low', 'close', 'diff', 'volume']]
            
        except Exception as e:
            print('Exception occured :', str(e))
            return None
        return df

    def replace_into_db(self, df, num, code, company):
        """네이버에서 읽어온 주식 시세를 DB에 REPLACE"""
        with self.conn.cursor() as curs:
            for r in df.itertuples():
                sql = f"REPLACE INTO daily_price VALUES ('{code}', "\
                    f"'{r.date}', {r.open}, {r.high}, {r.low}, {r.close}, "\
                    f"{r.diff}, {r.volume})"
                curs.execute(sql)
            self.conn.commit()
            print('[{}] #{:04d} {} ({}) : {} rows > REPLACE INTO daily_'\
                'price [OK]'.format(datetime.now().strftime('%Y-%m-%d'\
                ' %H:%M'), num+1, company, code, len(df)))

    def update_daily_price(self, pages_to_fetch):
        """KRX 상장법인의 주식 시세를 네이버로부터 읽어서 DB에 업데이트"""  
        for idx, code in enumerate(self.codes):
            df = self.read_naver(code, self.codes[code], pages_to_fetch)
            if df is None:
                continue
            self.replace_into_db(df, idx, code, self.codes[code])            

    def execute_daily(self):
        """실행 즉시 및 매일 오후 다섯시에 daily_price 테이블 업데이트"""
        pages_to_fetch = 2
        self.update_comp_info()
        self.update_daily_price(pages_to_fetch)
        
if __name__ == '__main__':
    dbu = DBUpdater()
    dbu.execute_daily()

OperationalError: (1049, "Unknown database 'investar'")